In [ ]:
%matplotlib inline
#%pylab
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mpc
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

import scipy
from scipy import interpolate
import pandas as pd
import itertools
from root_numpy import root2array, root2rec, tree2rec, array2root
from ROOT import TChain

In [ ]:
def open_df(fname,treename="flashtrigger/opflash_opflash_tree",start=None,stop=None):
    return pd.DataFrame( root2array(fname, treename,start=start,stop=stop) )

In [ ]:
#
# Data loading
#
rate=5
minutes=0
hour=0
day=1

#num_cosmic = 5*60*(minutes+hour*60+day*60*24)
#num_bnb = int(num_cosmic/670)+1

#print 'need',num_cosmic,'cosmic'
#print 'need',num_bnb,'bnb'

df_mc_cosmic = open_df('flash_trigger_ana/cosmic.root').query('100<dt and dt<3200')
df_data_cosmic = open_df('flash_trigger_ana/data_cosmic.root').query('100<dt and dt<3200')
#df_bnb = open_df('flash_trigger_ana/bnb.root').query('0<dt and dt<1.6')



In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

#ax.set_yscale('log')
#plt.figure(figsize=(12,8))
#plt.yscale('log')

data_cosmic_high = df_data_cosmic.query('pe_total<2000')
mc_cosmic_high = df_mc_cosmic.query('pe_total<2000')
bnb_high = df_bnb.query('pe_total<2000')

weights_data_cosmic = [1./len(data_cosmic_high)] * len(data_cosmic_high)
weights_mc_cosmic = [1./len(mc_cosmic_high)] * len(mc_cosmic_high)
weights_bnb = [1./len(bnb_high)] * len(bnb_high)

plt.hist(bnb_high['pe_total'].ravel(),
        bins=100,range=(0,2000),weights=weights_bnb,
        label='MCC 6.1 BNB',alpha=0.5,color='magenta')
plt.hist(mc_cosmic_high['pe_total'].ravel(),
        bins=100,range=(0,2000),weights=weights_mc_cosmic,
        label='MCC 6.1 Cosmics',alpha=1.0,color='cyan')
plt.hist(data_cosmic_high['pe_total'].ravel(),
        bins=100,range=(0,2000),weights=weights_data_cosmic,
        label='Data Cosmics (Run 2377)',alpha=0.8,color='blue')
ax.legend(prop={'size':20})
plt.ylim(0,0.03)
plt.xlabel('OpFlash Photo-Electron Counts',fontsize=20)
plt.ylabel('OpFlash Fraction',fontsize=20)
plt.tick_params(labelsize=20)
plt.grid()
plt.title('OpFlash Photo-Electron Distribution (Area Normalized)',fontsize=20)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

#ax.set_yscale('log')
#plt.figure(figsize=(12,8))
#plt.yscale('log')

data_cosmic_low = df_data_cosmic.query('pe_total<500')
mc_cosmic_low = df_mc_cosmic.query('pe_total<500')
bnb_low = df_bnb.query('pe_total<500')

weights_data_cosmic = [1./len(data_cosmic_low)] * len(data_cosmic_low)
weights_mc_cosmic = [1./len(mc_cosmic_low)] * len(mc_cosmic_low)
weights_bnb = [1./len(bnb_low)] * len(bnb_low)

plt.hist(bnb_low['pe_total'].ravel(),
        bins=100,range=(0,500),weights=weights_bnb,
        label='MCC 6.1 BNB',alpha=0.5,color='magenta')
plt.hist(mc_cosmic_low['pe_total'].ravel(),
        bins=100,range=(0,500),weights=weights_mc_cosmic,
        label='MCC 6.1 Cosmics',alpha=1.0,color='cyan')
plt.hist(data_cosmic_low['pe_total'].ravel(),
        bins=100,range=(0,500),weights=weights_data_cosmic,
        label='Data Cosmics (Run 2377)',alpha=0.8,color='blue')
ax.legend(prop={'size':20})
plt.ylim(0,0.03)
plt.xlabel('OpFlash Photo-Electron Counts',fontsize=20)
plt.ylabel('OpFlash Fraction',fontsize=20)
plt.tick_params(labelsize=20)
plt.grid()
plt.title('OpFlash Photo-Electron Distribution (Area Normalized)',fontsize=20)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

#ax.set_yscale('log')
#plt.figure(figsize=(12,8))
#plt.yscale('log')

data_nevents = len(df_data_cosmic.groupby(['run','subrun','event']))
mc_nevents = len(df_mc_cosmic.groupby(['run','subrun','event']))

if mc_nevents < data_nevents: raise Exception
    
weights_mc_cosmic = [float(data_nevents) / float(mc_nevents)] * len(df_mc_cosmic)

plt.hist(df_data_cosmic['pe_total'].ravel(),
        bins=100,range=(0,5000),
        label='Data Cosmics (Run 2377)',alpha=1.0,color='blue')
plt.hist(df_mc_cosmic['pe_total'].ravel(),
        bins=100,range=(0,5000),weights=weights_mc_cosmic,
        label='MCC 6.1 Cosmics',alpha=0.8,color='red')
ax.legend(prop={'size':20})
plt.ylim(1,40000)
plt.xlabel('OpFlash Photo-Electron Counts',fontsize=20)
plt.ylabel('OpFlash Fraction',fontsize=20)
plt.tick_params(labelsize=20)
plt.grid()
plt.title('OpFlash Photo-Electron Distribution',fontsize=20)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

#ax.set_yscale('log')
#plt.figure(figsize=(12,8))
#plt.yscale('log')

data_nevents = len(df_data_cosmic.groupby(['run','subrun','event']))
mc_nevents = len(df_mc_cosmic.groupby(['run','subrun','event']))

if mc_nevents < data_nevents: raise Exception
    
weights_mc_cosmic = [float(data_nevents) / float(mc_nevents)] * len(df_mc_cosmic)

mc_mod = df_mc_cosmic['pe_total'].ravel()
mc_mod *= 1.55

plt.hist(df_data_cosmic['pe_total'].ravel(),
        bins=100,range=(0,5000),
        label='Data Cosmics (Run 2377)',alpha=1.0,color='blue')
plt.hist(mc_mod,
        bins=100,range=(0,5000),weights=weights_mc_cosmic,
        label='MCC 6.1 Cosmics (1.55 PE scale factor)',alpha=0.8,color='red')
ax.legend(prop={'size':20})
plt.ylim(0,40000)
plt.xlabel('OpFlash Photo-Electron Counts',fontsize=20)
plt.ylabel('OpFlash Fraction',fontsize=20)
plt.tick_params(labelsize=20)
plt.grid()
plt.title('OpFlash Photo-Electron Distribution',fontsize=20)
plt.show()